# Naive-Bayes (DMC 2003, SPAM-Erkennung)

## Metriken

Satz von Bayes:

$$
P(A|B) = \frac{P(A) \cdot P(B | A)}{P(B)}
$$

Klasse $C_k$ in Abhängigkeit der Features $X = (x_1,\,\dots,\,x_n)$:

$$
\begin{align}
P(C_k|X) &= \frac{P(C_k) \cdot P(X|C_k)}{P(X)} \\
&= \frac{P(C_k) \cdot P(x_1 \land x_2 \dots \land x_n | C_k)}{P(x_1 \land x_2 \dots \land x_n)}\\
&= \alpha \cdot P(C_k) \cdot P(x_1 \land x_2 \dots \land x_n | C_k),\quad \alpha\; \text{const}
\end{align}
$$

»Naive« Annahme: alle $x_i$ stochastisch unabhängig.

$$
P(C_k|X) =  \alpha \cdot P(C_k) \cdot \prod_{i=1}^n P(x_i|C_k)
$$

Wahrscheinlichste Klasse:

$$
\hat{k} = \text{argmax}_k P(C_k)\prod_i P(x_i|C_k)
$$

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing
from sklearn import naive_bayes, metrics, linear_model, dummy

Einlesen Data Frame, Spaltentrenner ist das Leerzeichen.

In [ ]:
dfData = pd.read_csv('data_dmc2003_train.txt', sep=' ')
dfData.info()

Analyse der Zielwerte.

In [ ]:
dfData.target.value_counts()

Aufteilen in Features und Zielgröße. Umwandeln der Zielgröße in Zahl. Aufteilen in Trainings- und Testmenge.

In [ ]:
dataFeatures = dfData.drop(['id', 'target'], axis=1).values
target = dfData.target.map({'no': 0, 'yes': 1}).values
Xtrain, Xtest, yTrain, yTest = model_selection.train_test_split(
    dataFeatures, target, test_size=0.3, random_state=23)
Xtrain.shape, yTrain.shape, Xtest.shape, yTest.shape, Xtrain.dtype

Untersuchen Wertebereich Features. Nur Werte von 0 und 1 treten auf.

In [ ]:
vals_train, counts_train = np.unique(Xtrain, return_counts=True)
print('Train: different values', vals_train, 'Counts', counts_train)
vals_test, counts_test = np.unique(Xtest, return_counts=True)
print('Test:  different values', vals_test, 'Counts', counts_test)


Test auf Fehlwerte.

In [ ]:
np.count_nonzero(np.isnan(Xtrain))

Naive-Bayes-Klassifikation. ``alpha`` ist ein Regulierungsparameter.

In [ ]:
classifier = naive_bayes.BernoulliNB()
classifier.fit(Xtrain, yTrain)
print('Train:', classifier.score(Xtrain, yTrain), 'Test:', classifier.score(Xtest, yTest))
predTest = classifier.predict(Xtest)
print(metrics.confusion_matrix(yTest, predTest))

## Erläuterung Confusion Matrix

~~~~
       Vorhersage
         0  1
-------------
GT: 0 | tn fp
    1 | fn tp ← Recall
            ↑
            Precision
~~~~

- Vorhergesagte positive Werte: tp + fp
- Tatsächliche positive Werte:  tp + fn
- Precision: tp/(tp + fp) – welcher Anteil der positiv vorhergesagten Werte ist wirklich positiv
- Recall:    tp/(tp + fn) – welcher Anteil der wirklich positiven Werte wurde als positiv vorhergesagt
- Entscheidungschwelle senken: mehr positive erkannt, Recall ↑, Precision ↓


Vorhersage von Wahrscheinlichkeiten. Shape ist (ZahlDatensätze, ZahlKlassen). Wir benötigen nur Klasse 1.

In [ ]:
probTest = classifier.predict_proba(Xtest)[:,1]
probTest

Einfluss der Entscheidungsschwelle. Nur Wahrscheinlichkeiten > Schwelle werden als positiv klassifiziert.

In [ ]:
threshold = 0.8
plt.scatter(yTest, probTest, c=(probTest >= threshold) == yTest, alpha=0.2)
plt.hlines(threshold, 0, 1)
plt.xlabel('GT')
plt.ylabel('Pred')
plt.xticks([0,1])
print('Oben: Precision, rechts: Recall')

Graphische Darstellung für verschiedene Schwellwerte.

In [ ]:
thresholds = np.linspace(0, 0.99, 50)
prec_rec = list()
for threshold in thresholds:
    predTest = probTest >= threshold
    precision = metrics.precision_score(yTest, predTest, zero_division=0)
    recall    = metrics.recall_score(yTest, predTest)
    f1 = metrics.f1_score(yTest, predTest)
    prec_rec.append(np.array([precision, recall, f1]))
prec_rec = np.stack(prec_rec)
prec_rec
plt.plot(thresholds, prec_rec[:,0], label='Precision')
plt.plot(thresholds, prec_rec[:,1], label='Recall')
plt.plot(thresholds, prec_rec[:,2], label='F1')
plt.legend()

Vergleich logistische Regression und Dummy-Classifier. Beide liefern Wahrscheinlichkeiten.

In [ ]:
clRegr = linear_model.LogisticRegression()
clRegr.fit(Xtrain, yTrain)
yProbRegr = clRegr.predict_proba(Xtest)[:,1]
yProbRegr
clDummy = dummy.DummyClassifier(strategy='uniform')
clDummy.fit(Xtrain, yTrain)
yProbDummy = clDummy.predict_proba(Xtest)[:,1]
yProbDummy

Confusion Matrix für logistische Regression

In [ ]:
yPredRegr = yProbRegr >= 0.5
metrics.confusion_matrix(yTest, yPredRegr)

… und für Dummy Classifier

In [ ]:
yPredDummy = yProbDummy >= 0.5
metrics.confusion_matrix(yTest, yPredDummy)

Receiver Operation Characteristics (ROC). Statt Precision und Recall werden False Positive Rate und True Positive Rate verwendet. Optimale Klassifikation: oberes Dreieck.

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(yTest, probTest)
plt.plot(fpr, tpr, label='Naive Bayes')
fpr, tpr, thresholds = metrics.roc_curve(yTest, yProbRegr)
plt.plot(fpr, tpr, label='Logistic Regression')
fpr, tpr, _ = metrics.roc_curve(yTest, yProbDummy)
plt.plot(fpr, tpr, label='Dummy')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()